# Adição de gênero aos dados da CAPES

- **Nota**: O encoding dos arquivos csv da CAPES é **Latin-1 (ISO-8859-1)**

In [ ]:
import pandas as pd
import requests
import json
import random
import time # Importa a biblioteca time para usar sleep
import os

In [ ]:
discentes = pd.read_excel('capes_discente_2021-2023.xlsx') #Coloque o nome do seu arquivo aqui

In [ ]:
discentes.columns

In [ ]:
primeiros_nomes = sorted({nome.strip().split(" ")[0] for nome in discentes['NM_DISCENTE']})

## Obtendo gênero a partir da API do IBGE

In [ ]:
# Definindo constantes para controle das requisições
REQUEST_TIMEOUT = 10  # Tempo limite em segundos para cada requisição
MAX_RETRIES = 3       # Número máximo de tentativas para cada sexo
INITIAL_BACKOFF = 2   # Tempo inicial de espera entre tentativas (em segundos)

def _get_frequencia_por_sexo(nome, sexo_param):
    """
    Função auxiliar para buscar a frequência total de um nome para um sexo específico
    com tentativas e backoff exponencial em caso de erro de conexão/timeout.
    """
    url = f"https://servicodados.ibge.gov.br/api/v2/censos/nomes/{nome}?sexo={sexo_param}"
    
    for attempt in range(MAX_RETRIES):
        try:
            print(f"  Tentativa {attempt + 1}/{MAX_RETRIES} para {nome.upper()} ({sexo_param})...")
            response = requests.get(url, timeout=REQUEST_TIMEOUT) 
            response.raise_for_status() # Levanta um erro para códigos de status HTTP 4xx/5xx
            data = response.json()

            total_frequencia = 0
            if data and isinstance(data, list) and len(data) > 0 and data[0].get("res"):
                for item_periodo in data[0]["res"]:
                    total_frequencia += item_periodo.get("frequencia", 0)
            
            return total_frequencia

        # Captura exceções específicas de timeout ou problemas de conexão
        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError) as e:
            print(f"  Erro de conexão/timeout para '{nome}' ({sexo_param}): {e}")
            if attempt < MAX_RETRIES - 1:
                sleep_time = INITIAL_BACKOFF * (2 ** attempt) + random.uniform(0, 1) # Backoff exponencial + jitter
                print(f"  Aguardando {sleep_time:.2f} segundos antes de re-tentar...")
                time.sleep(sleep_time)
            else:
                print(f"  Número máximo de tentativas ({MAX_RETRIES}) atingido para '{nome}' ({sexo_param}).")
                return 0
        except requests.exceptions.RequestException as e:
            # Para outros erros de requisição que não sejam timeout/conexão
            print(f"  Erro de Requisição para '{nome}' ({sexo_param}): {e}")
            return 0 # Não re-tentar para erros que não sejam de rede/timeout
        except json.JSONDecodeError:
            print(f"  Erro ao decodificar JSON para '{nome}' ({sexo_param}).")
            return 0
        except Exception as e:
            print(f"  Erro inesperado em _get_frequencia_por_sexo para '{nome}' ({sexo_param}): {e}")
            return 0
    return 0 # Retorna 0 se todas as tentativas falharem

# A função determinar_genero_nome permanece a mesma, pois as retentativas estão na auxiliar.
def determinar_genero_nome(nome):
    """
    Busca um nome na API do IBGE, soma as frequências por sexo
    e determina o gênero predominante baseado na frequência feminina.
    Atribui 'Desconhecido' se o nome não for encontrado ou não tiver dados.
    Se a frequência feminina for exatamente 0.5, atribui o gênero aleatoriamente.
    """
    print(f"Iniciando busca para o nome: {nome.upper()}...")

    total_feminino = _get_frequencia_por_sexo(nome, "F")
    total_masculino = _get_frequencia_por_sexo(nome, "M")

    total_geral = total_feminino + total_masculino

    if total_geral == 0:
        print(f"\n--- Resultado para '{nome.upper()}' ---")
        print(f"Nome '{nome}' não encontrado ou sem dados de frequência para nenhum sexo na API do IBGE.")
        print("**Gênero atribuído: Desconhecido.**")
        print("----------------------------------\n")
        return "Desconhecido", None, (0, 0)

    frequencia_feminina = total_feminino / total_geral

    if frequencia_feminina > 0.5:
        genero_atribuido = "Feminino"
    elif frequencia_feminina < 0.5:
        genero_atribuido = "Masculino"
    else: # frequencia_feminina == 0.5
        genero_atribuido = random.choice(["Feminino", "Masculino"])
        print(f"***Atenção: Frequência Feminina é exatamente 0.5. Gênero atribuído ALEATORIAMENTE.***")
    
    print(f"\n--- Resultado para '{nome.upper()}' ---")
    print(f"Total Feminino (Todas as décadas): {total_feminino}")
    print(f"Total Masculino (Todas as décadas): {total_masculino}")
    print(f"Total Geral: {total_geral}")
    print(f"Frequência Feminina (Total): {frequencia_feminina:.4f}")
    print(f"**Gênero Atribuído: {genero_atribuido}**")
    print("----------------------------------\n")

    return genero_atribuido, frequencia_feminina, (total_feminino, total_masculino)

# --- Testando a função com as novas regras de re-tentativa e backoff ---
determinar_genero_nome("CHARLYNE") 
determinar_genero_nome("Maria")
determinar_genero_nome("José")
determinar_genero_nome("NomeInexistenteTotal")

In [ ]:
nomes_genero = {}

for nome in primeiros_nomes:
    resultado_completo = determinar_genero_nome(nome)
    genero = resultado_completo[0]
    nomes_genero[nome] = genero

In [ ]:
nomes_genero

## Obtendo gênero a partir de dados baixados localmente ([Brasil IO](https://brasil.io/dataset/genero-nomes/grupos/))

In [ ]:
#Link de download do arquivo 'grupos.csv' do dataset 'Gênero dos nomes'
url_gz_file = "https://data.brasil.io/dataset/genero-nomes/grupos.csv.gz"

# Cabeçalhos para simular uma requisição de navegador
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
}


# Passa os cabeçalhos através do parâmetro storage_options
grupos = pd.read_csv(url_gz_file, storage_options=headers)

In [ ]:
grupos

In [ ]:
#Is 'name' always within 'names'?

# Function to check if a value is in a pipe-separated string
def is_value_in_split_column(main_value, pipe_separated_string):
    if pd.isna(main_value) or pd.isna(pipe_separated_string):
        return False # Handle NaN values

    # Split the string by '|' and create a list
    sub_category_list = [item.strip() for item in pipe_separated_string.split('|')]
    
    # Check if the main_value is in the list of sub_categories
    return main_value in sub_category_list

# axis=1 means apply the function to each row, so we can access columns by name
grupos['Is_Main_In_Sub'] = grupos.apply(
    lambda row: is_value_in_split_column(row['name'], row['names']), 
    axis=1
)

print(grupos['Is_Main_In_Sub'].all()) #'name' is always within 'names', so we can just use the 'names' and 'classification' columns

### Criando um dicionario com nome e classificacao

In [ ]:
grupos_generos = grupos[['classification', 'names']]
grupos_generos['list_names'] = grupos_generos['names'].str.split('|').apply(lambda x: [name.strip() for name in x if name.strip()])
#split separa tudo com '|'
#strip remove strings vazias e eventuais espaços

grupos_generos

In [ ]:
grupos_exploded = grupos_generos.drop(columns='names').explode('list_names')

grupos_exploded

In [ ]:
dicionario_generos = grupos_exploded.set_index('list_names')['classification'].to_dict()

In [ ]:
dicionario_generos

#### Salvando dicionário como arquivo json

In [ ]:
arquivo_json = 'dicionario_generos.json'

if os.path.exists(arquivo_json):
    print(f"Erro: O arquivo '{arquivo_json}' já existe. Nenhuma alteração foi feita.")
else:
    try:
        with open(arquivo_json, 'w', encoding='utf-8') as f:
            json.dump(dicionario_generos, f, ensure_ascii=False, indent=4)
            print(f"Dicionário salvo com sucesso em '{arquivo_json}'.")
    except Exception as e:
        print(f"Ocorreu um erro ao salvar o dicionário como JSON: {e}")

### Usando o dicionario para atribuir genero aos dados da CAPES

In [ ]:
# Função para criar dicionario de generos baseado no arquivo 'grupos.csv' do Brasil IO
def criar_dicionario_generos(grupos_csv_filepath: str) -> dict:
    grupos = pd.read_csv(grupos_csv_filepath)[['classification', 'names']] #Pegando só colunas relevantes do arquivo csv
    grupos['list_names'] = grupos['names'].str.split('|').apply(lambda x: [name.strip() for name in x if name.strip()]) #split separa tudo com '|', comprehension remove strings vazias e strip remove eventuais espaços
    grupos_exploded = grupos.drop(columns='names').explode('list_names') #Pega a lista de nomes e a expande (explode) para um formato mais comprido
    dicionario_generos = grupos_exploded.set_index('list_names')['classification'].to_dict() #Gerando dicionário de generos com base no primeiro nome
    return dicionario_generos


# Define a função auxiliar que será aplicada a cada nome completo
def genero_baseado_no_primeiro_nome(nome_completo: str,
                    dicionario_generos: dict) -> str:
    
    # Garante que é uma string, útil para lidar com NaNs ou outros tipos
    nome_completo_str = str(nome_completo) 
    
    primeiro_nome = nome_completo_str.split(' ')[0].strip().upper()
    
    # Usa .get() para retornar 'D' (Desconhecido) se o nome não for encontrado
    return dicionario_generos.get(primeiro_nome, 'D')


def adicionar_coluna_genero(df,
                                 coluna_nome_completo: str, 
                                 dicionario_generos: dict,
                                 coluna_genero: str,
                       ):
    """
    Extrai o primeiro nome de uma coluna, consulta um dicionário de gêneros baseados em primeiros nomes
    e retorna os gêneros correspondentes em uma nova coluna, usando df.apply().

    Args:
        df (pd.DataFrame): O DataFrame de entrada.
        coluna_nome_completo (str): O nome da coluna no DataFrame que contém os nomes completos.
        dicionario_generos (dict): Um dicionário onde as chaves são os primeiros nomes (em maiúsculas)
                                   e os valores são os gêneros ('F', 'M', 'Desconhecido', etc.).
        coluna_genero (str): Nome da nova coluna com o genero inferido com base no primeiro nome,

    Returns:
        pd.DataFrame: O DataFrame original com uma nova coluna .
    """
    df[coluna_genero] = df[coluna_nome_completo].apply(genero_baseado_no_primeiro_nome, 
                                                       dicionario_generos=dicionario_generos)
    
    return df

#### Genero discentes

In [ ]:
dicionario_generos = criar_dicionario_generos('grupos.csv')

In [ ]:
discentes_com_genero = adicionar_coluna_genero(discentes,
                                               'NM_DISCENTE',
                                               dicionario_generos,
                                               'GN_DISCENTE')

In [ ]:
discentes_com_genero[['NM_DISCENTE','GN_DISCENTE']]

In [ ]:
discentes_com_genero['GN_DISCENTE'].value_counts()

#### Genero docentes

In [ ]:
docentes =  pd.read_excel('capes_docentes.xlsx')

In [ ]:
docentes.columns

In [ ]:
docentes_com_genero = adicionar_coluna_genero(docentes,
                                               'NM_DOCENTE',
                                               dicionario_generos,
                                               'GN_DOCENTE')

In [ ]:
docentes_com_genero[['NM_DOCENTE', 'GN_DOCENTE']]

In [ ]:
docentes_com_genero['GN_DOCENTE'].value_counts()